In [1]:
import requests
from bs4 import BeautifulSoup 
from datetime import datetime                            
import pandas as pd 
import time 
import urllib.request 
from selenium.webdriver import Chrome
import json
import re     
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import datetime as dt
    

In [2]:
company_list = ['035420','001040']

In [3]:
def stock_crawler(code):
    
    browser  = Chrome()
    browser.maximize_window()
    
    name = code
    base_url = 'https://finance.naver.com/item/coinfo.nhn?code='+ name + '&target=finsum_more'
    
    browser.get(base_url)
   
    browser.switch_to_frame(browser.find_element_by_id('coinfo_cp'))
    
    browser.find_elements_by_xpath('//*[@class="schtab"][1]/tbody/tr/td[3]')[0].click()

    html0 = browser.page_source
    html1 = BeautifulSoup(html0,'html.parser')
    
    #기업명 뽑기
    title0 = html1.find('head').find('title').text
    print(title0.split('-')[-1])
    
    html22 = html1.find('table',{'class':'gHead01 all-width','summary':'주요재무정보를 제공합니다.'})
    
    #date scrapy
    thead0 = html22.find('thead')
    tr0 = thead0.find_all('tr')[1]
    th0 = tr0.find_all('th')
    
    date = []
    for i in range(len(th0)):
        date.append(''.join(re.findall('[0-9/]',th0[i].text)))
    
    #columns scrapy
    tbody0 = html22.find('tbody')
    tr0 = tbody0.find_all('tr')
    
    col = []
    for i in range(len(tr0)):

        if '\xa0' in tr0[i].find('th').text:
            tx = re.sub('\xa0','',tr0[i].find('th').text)
        else:
            tx = tr0[i].find('th').text

        col.append(tx)
    
    #main text scrapy
    td = []
    for i in range(len(tr0)):
        td0 = tr0[i].find_all('td')
        td1 = []
        for j in range(len(td0)):
            if td0[j].text == '':
                td1.append('0')
            else:
                td1.append(td0[j].text)

        td.append(td1)
    
    td2 = list(map(list,zip(*td)))
    
    df = pd.DataFrame(td2,columns = col,index = date)
    return df

In [4]:
df_first_row = stock_crawler('005930')

<ipython-input-3-d1cf19aef835>:12: DeprecationWarning: use driver.switch_to.frame instead
  browser.switch_to_frame(browser.find_element_by_id('coinfo_cp'))


 기업개요(삼성전자)


In [5]:
df_first_row

,매출액,영업이익,영업이익(발표기준),세전계속사업이익,당기순이익,당기순이익(지배),당기순이익(비지배),자산총계,부채총계,자본총계,...,부채비율,자본유보율,EPS(원),PER(배),BPS(원),PBR(배),현금DPS(원),현금배당수익률,현금배당성향(%),발행주식수(보통주)
2016/12,"2,018,667","292,407","292,407","307,137","227,261","224,157","3,104","2,621,743","692,113","1,929,630",...,35.87,"22,004.14","2,735",13.18,"26,636",1.35,570,1.58,17.81,"7,033,966,850"
2017/12,"2,395,754","536,450","536,450","561,960","421,867","413,446","8,422","3,017,521","872,607","2,144,914",...,40.68,"24,536.12","5,421",9.40,"30,427",1.67,850,1.67,14.09,"6,454,924,700"
2018/12,"2,437,714","588,867","588,867","611,600","443,449","438,909","4,540","3,393,572","916,041","2,477,532",...,36.97,"27,531.92","6,024",6.42,"35,342",1.09,"1,416",3.66,21.92,"5,969,782,550"
2019/12,"2,304,009","277,685","277,685","304,322","217,389","215,051","2,338","3,525,645","896,841","2,628,804",...,34.12,"28,856.02","3,166",17.63,"37,528",1.49,"1,416",2.54,44.73,"5,969,782,550"
2020/12,"2,368,070","359,939","359,939","363,451","264,078","260,908","3,170","3,782,357","1,022,877","2,759,480",...,37.07,"30,692.79","3,841",21.09,"39,406",2.06,"2,994",3.70,77.95,"5,969,782,550"
2021/12,"2,628,668","474,856",0,"489,140","355,642","351,325",0,"4,032,866","1,082,646","2,950,220",...,36.70,0,"5,172",16.55,"42,187",2.03,"1,799",2.10,30.58,0
2022/12,"2,890,403","612,367",0,"630,110","457,319","452,050",0,"4,387,738","1,137,031","3,265,708",...,34.82,0,"6,655",12.86,"46,632",1.84,"2,010",2.35,26.54,0
2023/12,"2,991,854","640,602",0,"663,899","485,805","479,702",0,"4,687,778","1,153,137","3,534,639",...,32.62,0,"7,062",12.12,"50,355",1.70,"2,204",2.58,27.43,0


In [6]:
for i in company_list :
    df_new = stock_crawler(i)
    df_first_row = pd.concat([df_first_row, df_new])
    
    df_all = df_first_row
    df_all.to_excel('tobinq.xlsx')

<ipython-input-3-d1cf19aef835>:12: DeprecationWarning: use driver.switch_to.frame instead
  browser.switch_to_frame(browser.find_element_by_id('coinfo_cp'))


 기업개요(NAVER)
 기업개요(CJ)
